# Querying a Pandas dataframe with a LLM

Here we build an AI agent which generates Python code, then run this code and returns the answer to the user.
This example uses `deepseek-r1:7b` locally, which does not always return answers in the desired format, so some requests end up crashing or needing extra formatting, but it can still perform remarkably well!

First we make a df, for instance by querying astronomical data from the Gaia mission:

In [1]:
%%time
from astroquery.gaia import Gaia

query_string = """SELECT l, b, phot_g_mean_mag, bp_rp, parallax, pmra, pmdec, parallax_error, pmra_error, pmdec_error
  FROM gaiadr3.gaia_source_lite
  WHERE random_index < 5000"""

job = Gaia.launch_job_async(query=query_string, verbose=False)
gaia_data = job.get_results()

INFO: Query finished. [astroquery.utils.tap.core]
CPU times: user 411 ms, sys: 63.4 ms, total: 474 ms
Wall time: 2.51 s


In [2]:
df_gaia = gaia_data.to_pandas()

In [3]:
df_gaia.shape

(5000, 10)

### Attempt using LlamaIndex

https://docs.llamaindex.ai/en/stable/examples/query_engine/pandas_query_engine/

Requires us to install `llama-index-experimental` and `llama-index-llms-ollama`.

LlamaIndex passes the result of `df.head()` to the LLM, along with a prompt asking to return an exectuable string.

DeepSeek seems too chatty for the default prompts in LlamaIndex, so the actual execution of the code providing the answer ends up crashing! Let's see how it should work:

In [4]:
import pandas as pd
from llama_index.experimental.query_engine import PandasQueryEngine

# Also specify that we use a local LLM through Ollama
from llama_index.llms.ollama import Ollama

llm_deepseek = Ollama(model="deepseek-r1:7b", request_timeout=120.0, temperature=0.0)

query_engine = PandasQueryEngine(df=df_gaia, verbose=True, llm=llm_deepseek)

In [5]:
%%time
response = query_engine.query(
    "How many rows does this dataframe contain?",
)

INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"


ReadTimeout: timed out

This failed because the LLM we used does not return output in the exact format expected by the LlamaIndex agent. 

We are going to do some manual coding to work around that.

### Building our agent from scratch

We pass some information about the df in the prompt, then ask the LLM to produce code. Sometimes I get code on the last line, most of the time I get it wrapped in triple backquotes. Sometimes I also get another sentence of text after the answer. Sometimes the code is correct but wrapped in a `print()` which makes it return a string. For reliable a consistent output, a more powerful model should ensure the output is always formatted properly.

In [6]:
import ollama


class PandasAgent:
    def __init__(self, df: pd.DataFrame):
        self.df = df

    def query(self, user_question: str, ollama_llm: str):
        df = self.df
        df_str = df.head()
        self.pandas_prompt = (
            "CONTEXT:\n"
            "You are working with a pandas dataframe in Python.\n"
            "The name of the dataframe is `df`.\n"
            "This is the result of `print(df.head())`:\n"
            f"{df_str}\n\n"
            "INSTRUCTIONS:\n"
            "Convert the user request to a one-liner executable Python code using Pandas.\n"
            "Reply with a single line."
            "The question is:\n"
            f"{user_question}\n"
            "If there are more than one way of solving it, just choose one.\n"
            "IMPORTANT:\n"
            "Your reply should not contain any text, only code."
        )
        self.full_output = ollama.generate(
            model=ollama_llm,
            prompt=self.pandas_prompt,
            options={"temperature": 0},
        )

        self.code_response = self.full_output.response.split("```python\n")[-1].split(
            "\n"
        )[0]
        # deal with print(...)
        if self.code_response.startswith("print("):
            self.code_response = self.code_response[6:-1]

        try:
            self.actual_result = eval(self.code_response)
            return self.actual_result
        except Exception:
            print(self.code_response, "cannot be evaluated.")
            return None

In [7]:
agent_df_gaia = PandasAgent(df_gaia)

In [8]:
%%time
answer = agent_df_gaia.query(
    "How many rows are there in the dataframe?", ollama_llm="deepseek-r1:7b"
)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
CPU times: user 5.98 ms, sys: 21.2 ms, total: 27.2 ms
Wall time: 28.4 s


In [9]:
print("The result is:", answer)
print(agent_df_gaia.pandas_prompt)
print(agent_df_gaia.full_output.response)

The result is: 5000
CONTEXT:
You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
            l          b  phot_g_mean_mag     bp_rp  parallax      pmra  \
0   34.939058  -3.306332        15.244129  1.522128  1.084924  2.549028   
1  286.532346  -6.648696        20.906347  0.702095  0.854356 -2.606806   
2  104.935184 -15.490523        20.531225  1.649284  1.042008 -1.729704   
3  346.096650   4.141483        20.145899  2.006773  0.587660 -3.566062   
4  344.190022  -4.748318        19.787357  1.573410 -0.293376  0.209898   

      pmdec  parallax_error  pmra_error  pmdec_error  
0 -4.075544        0.033431    0.030442     0.026224  
1  4.414332        1.282721    2.147447     1.408533  
2 -3.353289        0.633289    0.514036     0.558126  
3 -4.132508        0.633092    1.053221     0.712860  
4 -3.433852        0.619885    0.773631     0.594055  

INSTRUCTIONS:
Convert the user request to a one-liner executa

It worked!!!

## Trying a more complex query

In [10]:
answer = agent_df_gaia.query(
    """Give the median parallax of the df rows grouped by integer value of their phot_g_mean_mag. 
Note that the column phot_g_mean_mag contains some NaN values.""",
    ollama_llm="deepseek-r1:7b",
)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


In [11]:
print("The result is:", answer)
# print(agent_df_gaia.pandas_prompt)
print(agent_df_gaia.full_output.response)

The result is: phot_g_mean_mag
10.0    1.570822
11.0    1.604811
12.0    0.848711
13.0    1.056185
14.0    0.659533
15.0    0.461134
16.0    0.377964
17.0    0.328263
18.0    0.258979
19.0    0.296725
20.0    0.323012
21.0         NaN
Name: parallax, dtype: float64
<think>
Okay, so I need to figure out how to calculate the median parallax for each group in the dataframe based on the integer value of phot_g_mean_mag. The user mentioned that phot_g_mean_mag has some NaN values, so I have to handle those.

First, I remember that pandas can group data using the groupby function. So I'll start by grouping the dataframe 'df' by the 'phot_g_mean_mag' column. But since there are NaNs, I should convert the values to integers after dropping the NaNs to ensure proper grouping.

Wait, how do I handle the NaNs? Maybe I can drop them first before converting to integer. So I'll use df.dropna(subset=['phot_g_mean_mag']) to exclude rows where phot_g_mean_mag is NaN. Then, I'll cast that column to integ

It worked! The LLM found the right solution AND returned the one-liner in a parsable format.

## Another complex query

In [12]:
answer = agent_df_gaia.query(
    """Is the mean value of l larger than the mean value of b?""",
    ollama_llm="deepseek-r1:7b",
)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


In [13]:
print("The result is:", answer)
# print(agent_df_gaia.pandas_prompt)
print(agent_df_gaia.full_output.response)

The result is: True
<think>
Okay, so I need to figure out how to determine if the mean value of 'l' is larger than the mean value of 'b' in the given pandas DataFrame. Let me break this down step by step.

First, I know that in pandas, calculating the mean of a column is straightforward using the .mean() method. So for each column 'l' and 'b', I can compute their respective means.

The user's question is asking whether the average of 'l' is greater than the average of 'b'. That translates to comparing these two means. If df['l'].mean() is greater than df['b'].mean(), then the answer is yes; otherwise, no.

I should write a one-liner that performs this comparison. So combining everything, it would be something like (df['l'] > df['b']).mean(). But wait, actually, I just need to check if the mean of 'l' is greater than the mean of 'b'. So comparing the two means directly.

Alternatively, another approach could be to subtract the mean of 'b' from the mean of 'l' and see if it's positive. T

Correct again! This time it forgot to return a single solution, and added a lot of text around the answers too, but our parsing scheme was still able to grab the executable snippet of code.